In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import pandas as pd
import glob,os
from sklearn.model_selection import train_test_split

Mounted at /content/drive


In [ ]:
#https://github.com/merry555/FibVID/tree/v0.0.1
#news_claim.csv : this file provides a list of news contents that include claim number, group, text, and news source.
news_claim = pd.read_csv('/content/drive/MyDrive/QE/Study2/FibVID-v0.0.1/dataset/news_claim/news_claim.csv')
news_claim

#All the data were grouped as follows: 0 as COVID True claims, 1 as COVID Fake claims, 2 as Non-COVID True claims, and 3 as Non-COVID Fake claims.

,text,source,claim_num,group
0,the (covid 19) cases are going up but it's bec...,politifact,0,1
1,every election year has a disease sars in 200...,politifact,1,1
2,obama ordered cia to train isis jihadists,politifact,2,3
3,"says george soros said, im going to bring down...",politifact,3,3
4,if you paid rent in march or april they got to...,politifact,4,1
...,...,...,...,...
1348,says a photo shows patrick mahomes wearing a s...,politifact,1348,3
1349,nancy pelosi may have just committed a violati...,politifact,1349,3
1350,"says donald trump tweeted, i have it on good a...",politifact,1350,3
1351,says eight iowa counties have more adults regi...,politifact,1351,3


In [ ]:
news_claim.group.value_counts()

group
1    569
3    431
0    203
2    150
Name: count, dtype: int64

In [ ]:
#origin_tweet.csv : this file provides assurance that the correct origin tweets were collected in reference to claims collected from Politifact and Snopes. The dataset includes tweet user, tweet id, retweet count, post text, hashtag, like count, create date, group, and similarity.

origin_data = pd.read_csv('/content/drive/MyDrive/QE/Study2/FibVID-v0.0.1/dataset/claim_propagation/origin_tweet.csv')
origin_data

,create_date,tweet_id,similarity,like_count,retweet_count,post_text,claim_number,group,hashtag,tweet_user
0,2020-07-28T14:04:36.000Z,1288113129889517578,0.000000,2823,1050,The latest round of testing from the Miami Mar...,0,1,NaN,85952
1,2020-07-28T03:36:37.000Z,1287955090184105984,0.000000,54,100,Scammers are cold calling Victorians claiming ...,0,1,COVID,5342
2,2020-07-27T21:05:06.000Z,1287856562200088581,0.000000,822,428,"“It feels like we’re alone,” Hialeah, FL, Mayo...",0,1,NaN,78787
3,2020-07-27T14:54:11.000Z,1287763218933395461,0.000000,36687,14982,The best laid plans of a multi-billion dollar ...,0,1,NaN,80798
4,2020-04-02T14:37:32.000Z,1245722037869477896,0.404285,13276,4154,NYC’s case fatality rate for Covid19 is now al...,57,1,NaN,80798
...,...,...,...,...,...,...,...,...,...,...
1769,2020-12-10T06:10:04.000Z,1336916069336948736,0.445844,2328,345,Struggling after speaking with ⁦@donlemon⁩ ton...,298,1,COVID19,61051
1770,2020-12-04T15:10:13.000Z,1334877674540572672,0.328623,11593,2454,As the dust settles on Democrats' downballot f...,298,1,NaN,42186
1771,2020-12-05T11:19:16.000Z,1335181945635004417,0.659882,1436,180,We broke the news on #OutLoudWithGiannoCaldwel...,298,1,OutLoudWithGiannoCaldwell,73973
1772,2020-12-20T20:31:55.000Z,1340756841454850049,0.390273,171932,2654,I see your tweets and I’m trying to double and...,298,1,NaN,75482


In [13]:
#claim_propagation.csv : this file provides the claim propagation that includes claim number, parent user, parent id, tweet user, tweet id, retweet count, post text, hashtag, like count, create date, group, depth.

claim_propagation = pd.read_csv('/content/drive/MyDrive/QE/Study2/FibVID-v0.0.1/dataset/claim_propagation/claim_propagation.csv').dropna(subset='group')
claim_propagation

,tweet_user,tweet_id,like_count,depth,parent_user,create_date,parent_id,retweet_count,post_text,claim_number,group,hashtag
0,64,1260590991641362436,301,0,0,2020-05-13T15:21:27.000Z,0,113,So you left a 69 page guidebook but no PPE or ...,141,1,NaN
1,135,1288771853822103552,194,0,0,2020-07-30T09:42:08.000Z,0,99,More tales from the NHS frontline are emerging...,138,0,NaN
2,290,1251209120528637952,7650,0,0,2020-04-17T18:01:15.000Z,0,4279,DEVELOPING: A crowd of protesters could be see...,30,0,NaN
3,361,1339567071588442113,621,0,0,2020-12-17T13:44:12.000Z,0,231,"Latest we expect on checks, *as of now*, per a...",285,3,NaN
4,457,1265997510738358272,959,0,0,2020-05-28T13:25:02.000Z,0,271,Wisconsin election officials agree to send vot...,35,3,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
221248,144736,1235937681047080960,0,1,106463,2020-03-06T14:38:00.000Z,1235932698688053249,0,"#WaterIsLife \nremember your vote matters, \nC...",229,2,WaterIsLife#WhenWeAllVote#FlintWaterCrisis
221249,144737,1279815303631798273,1,1,141002,2020-07-05T16:32:00.000Z,1279489733249433600,1,don’t go get tested. i’m sure you will beat so...,131,1,NaN
221250,144738,1242264347025571842,0,1,93248,2020-03-24T01:37:55.000Z,1242257594540056576,1,NO @GOP should even entertain this anti-Americ...,96,0,NaN
221251,144739,1234597235746394118,0,1,56172,2020-03-02T21:51:33.000Z,1234592892896923652,0,I assume if you win it will the begging of the...,173,3,NaN


In [14]:
claim_propagation.group.value_counts()

group
1    133374
3     47790
0     27296
2     12793
Name: count, dtype: int64

In [15]:
claim_propagation[claim_propagation.parent_id==0].group.value_counts()

group
1    898
3    483
0    270
2    123
Name: count, dtype: int64

In [ ]:
#user_information.csv : this file provides the user information that includes user id, create date, description, follower count, and the following count.
user_information = pd.read_csv('/content/drive/MyDrive/QE/Study2/FibVID-v0.0.1/dataset/user_information/user_information.csv')
user_information

,following_count,follower_count,creation_date,description,user_id
0,101.0,56.0,2020-03-26 15:57:20,NaN,0
1,8104.0,22533.0,2015-07-21 22:10:47,"People say I’m skeptical, but I don’t believe ...",1
2,598.0,372.0,2019-02-02 19:01:35,Motorcycles. Retired merchant Mariner Captain...,2
3,322.0,203.0,2020-04-01 07:32:57,"Cheese, wine, medieval history, licensed foren...",3
4,102.0,4.0,2020-11-07 13:23:04,NaN,4
...,...,...,...,...,...
144736,4674.0,3338.0,2019-12-06 23:54:47,🧢 #YangGangForever\n #YangIt☯️\n.\n.\n(backing...,144736
144737,91.0,405.0,2010-06-23 02:49:46,Overly Blunt Podcast and CHRISPIRACY.,144737
144738,7416.0,6753.0,2017-05-30 16:34:29,Retired RN ~ Mom enjoying family & the #Saltli...,144738
144739,284.0,18765.0,2012-04-16 22:32:36,🇰🇼كل ما أكتب من وحي قلمي على صفحات أيامي المؤق...,144739
